In [1]:
import json
import pathlib
import urllib.parse

import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests
import shapely
import sqlalchemy as db
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
import subprocess

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from shapely.geometry import Point
import geopandas as gpd
import psycopg2

/Users/alexisabdelnour/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/alexisabdelnour/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/Users/alexisabdelnour/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## <b> Part 1 </b>: Data Preprocessing

### Important Note on zipcode dataset

The folder downloaded contains many files, but here we want to import the .shp one. To understand the others, please note:
'.shp' - the main file that stores the geometry of all features.

'.shx' - the index file that allows for fast searching and querying of the geometry. 

'.dbf' - the attribute format file that stores tabular data for each feature.

'.prj' - the projection file that stores the coordinate system and projection information.

'.sbn' and .sbx - files that store the spatial index of the features.

'.shp.xml" - the metadata file that stores information about the shapefile itself.

#### Reading in the APP Token from a secrets,json file

In [2]:
#Read in the app token id
with open("secrets.json", "r") as file:
    secrets = json.load(file)

#### Setting up global variables we will be needing everywhere

In [3]:
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "nyc_zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "FaZUKnHbC1w74WJvH3vnz8br8"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/resource/"
NYC_DATA_311 = "erm2-nwe9.json"
NYC_DATA_TREES = "5rq2-4hqu.json"

DB_NAME = "group8project"
DB_USER = "alexisabdelnour" #figure out your username by running 'psql' command
DB_URL = f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}"

DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

## 1. For the Zillow data

#### Read-in Zillow Data

In [4]:
df_zillow_data = pd.read_csv(ZILLOW_DATA_FILE)

In [5]:
df_zillow_data

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1471.214336,...,1843.953065,1853.546220,1860.805060,1873.335787,1879.080480,1882.092604,1877.636803,1857.636589,1846.701735,1839.654960
1,61148,2,8701,zip,NJ,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1772.500000
2,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1285.448996,...,1799.232097,1803.978538,1815.603187,1824.661645,1837.338997,1837.127291,1822.343233,1809.231267,1813.118556,1830.410884
3,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,NaN,...,1841.080353,1859.306448,1864.888023,1860.577001,1864.587265,1861.155500,1889.310623,1879.227108,1886.003582,1881.000505
4,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,NaN,...,1373.506118,1378.743001,1381.945632,1405.219882,1403.055653,1409.571610,1414.197679,1418.818592,1434.125145,1430.949495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6627,418163,30158,89158,zip,NV,NV,Las Vegas,"Las Vegas-Henderson-Paradise, NV",Clark County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2970.805951,3037.250000
6628,72017,30490,32461,zip,FL,FL,Panama City Beach,"Crestview-Fort Walton Beach-Destin, FL",Walton County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3440.933177,3545.000000
6629,91179,30490,76005,zip,TX,TX,Arlington,"Dallas-Fort Worth-Arlington, TX",Tarrant County,NaN,...,2039.546586,2051.648737,2079.625377,2056.221398,2055.176610,1989.931670,1945.129320,1882.124514,1888.357906,1862.293651
6630,61618,30490,10004,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,NaN,...,5214.029414,5241.715117,5303.140730,5266.321434,5361.400722,5366.014683,5425.512870,5307.152249,5063.896586,5030.416667


### 1.1 Removing Unnecessary columns
First part is to remove unnecessary columns and keep only the ones relevant to answer this question. Since our main objective is to find a quiet neighborhood with a lot of greenery within the budget, the most important columns are the ZIP code (RegionName), possibly the city or county name for additional geographic filtering, and the rent averages. The RegionID, SizeRank, RegionType, and StateName columns might be less relevant so we can remove them

In [6]:
# Identify columns to keep: geographic identifiers + columns from 2023 onwards
columns_to_keep = ['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName']
columns_to_keep += [col for col in df_zillow_data.columns if col.startswith('2023') or col.startswith('2024')]

# Filter the DataFrame to keep only the selected columns
df_zillow_data = df_zillow_data[columns_to_keep]

### 1.2 Renaming columns for clarity
Renaming columns can help make the dataset more understandable at a glance. The format here is consitant but the region name can be renamed as ZIP code for clarity

In [7]:
# Renaming columns for clarity
df_zillow_data.rename(columns={'RegionName': 'zip_code'}, inplace=True)
    
# Create a mapping from original column names to new names
column_name_mapping = {
        '2023-01-31': 'Jan_2023',
        '2023-02-28': 'Feb_2023',
        '2023-03-31': 'Mar_2023',
        '2023-04-30': 'Apr_2023',
        '2023-05-31': 'May_2023',
        '2023-06-30': 'Jun_2023',
        '2023-07-31': 'Jul_2023',
        '2023-08-31': 'Aug_2023',
        '2023-09-30': 'Sep_2023',
        '2023-10-31': 'Oct_2023',
        '2023-11-30': 'Nov_2023',
        '2023-12-31': 'Dec_2023',
        '2024-01-31': 'Jan_2024'
    }

/var/folders/4x/r3yt7kvn0c1_gwb2rfp7_2vm0000gn/T/ipykernel_44228/2642397087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_zillow_data.rename(columns={'RegionName': 'zip_code'}, inplace=True)


### 1.3 Handling Missing Data
There are many ways to handle missing data. Either we can add a default hard code filling them with the average rent of nearby months, we can also drop the entire row when it contains missing data, or fill missing values based on surrounding data.

To evaluate which method is the best, we can see how many rows contain missing values, and if by dropping we will not remove a too big chunk of the data

In [8]:
# Step 1: Check for missing data
missing_data = df_zillow_data.isnull()

# Step 2: Count missing values per column
missing_per_column = missing_data.sum()
print("Missing values per column:\n", missing_per_column)

# Step 3: Count rows with at least one missing value
rows_with_missing = missing_data.any(axis=1).sum()
print(f"Total rows with at least one missing value: {rows_with_missing}")

Missing values per column:
 RegionID         0
SizeRank         0
zip_code         0
RegionType       0
StateName        0
State            0
City            60
Metro           11
CountyName       0
2023-01-31    2057
2023-02-28    1972
2023-03-31    1803
2023-04-30    1708
2023-05-31    1593
2023-06-30    1525
2023-07-31    1501
2023-08-31    1443
2023-09-30    1402
2023-10-31    1292
2023-11-30    1160
2023-12-31     889
2024-01-31       1
dtype: int64
Total rows with at least one missing value: 2238


We can see that there are 2238 rows with at least one missing value and it would be too. The file is 6632 rows so we would be deleting most of it. We looked at different methods to handle those missing values, and used an iterative imputation algorithm that models each feature as a function of other features, where each feature is imputed sequentially. 

In [9]:
#Handling missing values
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Create a list of the columns with missing values
cols_with_missing_values = ['2023-01-31','2023-02-28',
                            '2023-03-31','2023-04-30','2023-05-31','2023-06-30','2023-07-31',
                            '2023-08-31','2023-09-30','2023-10-31','2023-11-30',
                            '2023-12-31','2024-01-31']

# Impute missing values using IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imputed_predictors = imp.fit_transform(df_zillow_data[cols_with_missing_values])

# Replace the original columns with the imputed columns
df_zillow_data[cols_with_missing_values] = imputed_predictors

/Users/alexisabdelnour/opt/anaconda3/lib/python3.9/site-packages/sklearn/impute/_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"
/var/folders/4x/r3yt7kvn0c1_gwb2rfp7_2vm0000gn/T/ipykernel_44228/1943664251.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_zillow_data[cols_with_missing_values] = imputed_predictors


In [10]:
# Checking the effect of the above method
df_zillow_data.isna().sum()

RegionID       0
SizeRank       0
zip_code       0
RegionType     0
StateName      0
State          0
City          60
Metro         11
CountyName     0
2023-01-31     0
2023-02-28     0
2023-03-31     0
2023-04-30     0
2023-05-31     0
2023-06-30     0
2023-07-31     0
2023-08-31     0
2023-09-30     0
2023-10-31     0
2023-11-30     0
2023-12-31     0
2024-01-31     0
dtype: int64

#### Let us make the column names more readable

In [11]:
column_name_mapping = {
    '2023-01-31': 'Jan_2023',
    '2023-02-28': 'Feb_2023',
    '2023-03-31': 'Mar_2023',
    '2023-04-30': 'Apr_2023',
    '2023-05-31': 'May_2023',
    '2023-06-30': 'Jun_2023',
    '2023-07-31': 'Jul_2023',
    '2023-08-31': 'Aug_2023',
    '2023-09-30': 'Sep_2023',
    '2023-10-31': 'Oct_2023',
    '2023-11-30': 'Nov_2023',
    '2023-12-31': 'Dec_2023',
    '2024-01-31': 'Jan_2024'
}

df_zillow_data.rename(columns=column_name_mapping, inplace=True)

/var/folders/4x/r3yt7kvn0c1_gwb2rfp7_2vm0000gn/T/ipykernel_44228/3029084057.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_zillow_data.rename(columns=column_name_mapping, inplace=True)


## 2. For the NYC Zipcode data

##### Read-in and clean NYC 311 Data

In [12]:
geodf_zipcode_data = gpd.read_file(ZIPCODE_DATA_FILE)

Here, in order to pre-process the data, we will start by checking its structure

In [13]:
# Check the GeoDataFrame's structure
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ZIPCODE     263 non-null    object  
 1   BLDGZIP     263 non-null    object  
 2   PO_NAME     263 non-null    object  
 3   POPULATION  263 non-null    float64 
 4   AREA        263 non-null    float64 
 5   STATE       263 non-null    object  
 6   COUNTY      263 non-null    object  
 7   ST_FIPS     263 non-null    object  
 8   CTY_FIPS    263 non-null    object  
 9   URL         263 non-null    object  
 10  SHAPE_AREA  263 non-null    float64 
 11  SHAPE_LEN   263 non-null    float64 
 12  geometry    263 non-null    geometry
dtypes: float64(4), geometry(1), object(8)
memory usage: 26.8+ KB


We can see the columns do not have all the same format in the naming so we can standardize this

### 2.1 Renaming columns for clarity

As in the last section, to ensure a smooth attribute name, we can rename as such:

In [14]:
geodf_zipcode_data.rename(columns={
    'ZIPCODE': 'zip_code',
    'BLDGZIP': 'building_zip',
    'PO_NAME': 'city',
    'POPULATION': 'population',
    'AREA': 'area',
    'STATE': 'state', 
    'COUNTY': 'county', 
    'ST_FIPS': 'state_fips_code',
    'CTY_FIPS': 'county_fips_code',
    'URL': 'info_url',
    'SHAPE_AREA': 'shape_area',
    'SHAPE_LEN': 'shape_perimeter'}, inplace=True)

### 2.2 Verify geometry
Verify that the geometries in the geometry column are valid and correct any issues. Invalid geometries can cause problems in spatial analyses and visualizations.

In [15]:
# Check for and attempt to fix any invalid geometries
geodf_zipcode_data['geometry'] = geodf_zipcode_data['geometry'].buffer(0)

### 2.3 Verify CRS
Ensure the GeoDataFrame uses the appropriate CRS for the analysis,it can be usefull if we perform spatial operations with other datasets.

In [16]:
# Set or verify the CRS (e.g., EPSG:4326 for WGS84 Latitude/Longitude)
print(geodf_zipcode_data.crs)
# Here it is 2263, so we convert
geodf_zipcode_data.to_crs(epsg=4326, inplace=True)

EPSG:2263


In [17]:
#check
print(geodf_zipcode_data.crs)

EPSG:4326


### 2.4 Handling Missing Data
Same as in the last section, we want to verify if there are a lot of missing data and potentially find a solution.

In [18]:
# Step 1: Check for missing data
missing_data = geodf_zipcode_data.isnull()

# Step 2: Count missing values per column
missing_per_column = missing_data.sum()
print("Missing values per column:\n", missing_per_column)

# Step 3: Count rows with at least one missing value
rows_with_missing = missing_data.any(axis=1).sum()
print(f"Total rows with at least one missing value: {rows_with_missing}")

Missing values per column:
 zip_code            0
building_zip        0
city                0
population          0
area                0
state               0
county              0
state_fips_code     0
county_fips_code    0
info_url            0
shape_area          0
shape_perimeter     0
geometry            0
dtype: int64
Total rows with at least one missing value: 0


As we can see here there is no NaN so we can skip this section.

# 3. For the NYC Open data

##### Read-in and clean NYC 311 Data

In [19]:
def download_and_clean_311_data():
    
    data_url = BASE_NYC_DATA_URL + NYC_DATA_311
    columns = "unique_key,created_date,closed_date,complaint_type,incident_zip,latitude,longitude"
    start_date = "2023-03-01"
    end_date = "2024-02-29"
    limit = 50000
    offset = 0
    total_rows = 0
    headers = {"X-App-Token": NYC_DATA_APP_TOKEN}
    all_data = []
    
    while True:
        query = f"$select={columns}&$where=created_date >= '{start_date}T00:00:00.000' AND created_date <= '{end_date}T23:59:59.999' AND latitude IS NOT NULL&$limit={limit}&$offset={offset}"
        paginated_url = f"{data_url}?{query}"
        response = requests.get(paginated_url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data:
                all_data.extend(data)
                total_rows += len(data)
                offset += limit
            else:
                break
        else:
            break
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    df = df.rename(columns={'incident_zip':'zip_code'})
    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326"
    )
    return gdf


In [20]:
geodf_311_data = download_and_clean_311_data()

##### Tree Data

In [21]:
def download_and_clean_tree_data():
    limit = 50_000
    offset = 0
    #columns = "tree_id,spc_common,zipcode,latitude,longitude"
    columns = "tree_id,spc_common,zipcode,latitude,longitude,status,health,steward,sidewalk"
    data_url = BASE_NYC_DATA_URL + NYC_DATA_TREES
    headers = {"X-App-Token": NYC_DATA_APP_TOKEN}
    all_data = []
    while True:
        query = f"$select={columns}&$limit={limit}&$offset={offset}"
        paginated_url = f"{data_url}?{query}"
        response = requests.get(paginated_url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data:
                all_data.extend(data)
                offset += limit
            else:
                break
        else:
            break
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    df = df.rename(columns={'zipcode':'zip_code'})
    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df['longitude'].astype(float), df['latitude'].astype(float)), crs="EPSG:4326"
    )
    return gdf

In [22]:
geodf_tree_data = download_and_clean_tree_data()

##### Getting basic info on all the tables

In [23]:
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   zip_code          263 non-null    object  
 1   building_zip      263 non-null    object  
 2   city              263 non-null    object  
 3   population        263 non-null    float64 
 4   area              263 non-null    float64 
 5   state             263 non-null    object  
 6   county            263 non-null    object  
 7   state_fips_code   263 non-null    object  
 8   county_fips_code  263 non-null    object  
 9   info_url          263 non-null    object  
 10  shape_area        263 non-null    float64 
 11  shape_perimeter   263 non-null    float64 
 12  geometry          263 non-null    geometry
dtypes: float64(4), geometry(1), object(8)
memory usage: 26.8+ KB


In [24]:
geodf_zipcode_data.head()

,zip_code,building_zip,city,population,area,state,county,state_fips_code,county_fips_code,info_url,shape_area,shape_perimeter,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."


In [25]:
geodf_311_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3236487 entries, 0 to 3236486
Data columns (total 8 columns):
 #   Column          Dtype   
---  ------          -----   
 0   unique_key      object  
 1   created_date    object  
 2   closed_date     object  
 3   complaint_type  object  
 4   zip_code        object  
 5   latitude        object  
 6   longitude       object  
 7   geometry        geometry
dtypes: geometry(1), object(7)
memory usage: 197.5+ MB


In [26]:
geodf_311_data.head()

,unique_key,created_date,closed_date,complaint_type,zip_code,latitude,longitude,geometry
0,60462274,2024-02-29T23:59:31.000,2024-03-01T00:00:00.000,Building/Use,11204,40.61984470790165,-73.98238922950797,POINT (-73.98239 40.61984)
1,60452687,2024-02-29T23:59:27.000,2024-03-02T10:47:34.000,HEAT/HOT WATER,11209,40.62326209885787,-74.03020533301675,POINT (-74.03021 40.62326)
2,60451322,2024-02-29T23:59:13.000,2024-03-01T00:48:14.000,Noise - Vehicle,11237,40.700045051717986,-73.91198085791687,POINT (-73.91198 40.70005)
3,60449904,2024-02-29T23:58:39.000,2024-03-01T01:13:49.000,Blocked Driveway,11429,40.713059105370455,-73.75260269888014,POINT (-73.75260 40.71306)
4,60452421,2024-02-29T23:58:22.000,2024-03-01T00:24:56.000,Noise - Commercial,11419,40.68414711447904,-73.83262819498826,POINT (-73.83263 40.68415)


In [27]:
geodf_tree_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   tree_id     683788 non-null  object  
 1   spc_common  652169 non-null  object  
 2   zip_code    683788 non-null  object  
 3   latitude    683788 non-null  object  
 4   longitude   683788 non-null  object  
 5   status      683788 non-null  object  
 6   health      652172 non-null  object  
 7   steward     652173 non-null  object  
 8   sidewalk    652172 non-null  object  
 9   geometry    683788 non-null  geometry
dtypes: geometry(1), object(9)
memory usage: 52.2+ MB


In [28]:
geodf_tree_data.head()

,tree_id,spc_common,zip_code,latitude,longitude,status,health,steward,sidewalk,geometry
0,180683,red maple,11375,40.72309177,-73.84421522,Alive,Fair,None,NoDamage,POINT (-73.84422 40.72309)
1,200540,pin oak,11357,40.79411067,-73.81867946,Alive,Fair,None,Damage,POINT (-73.81868 40.79411)
2,204026,honeylocust,11211,40.71758074,-73.9366077,Alive,Good,1or2,Damage,POINT (-73.93661 40.71758)
3,204337,honeylocust,11211,40.71353749,-73.93445616,Alive,Good,None,Damage,POINT (-73.93446 40.71354)
4,189565,American linden,11215,40.66677776,-73.97597938,Alive,Good,None,Damage,POINT (-73.97598 40.66678)


In [29]:
df_zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6632 entries, 0 to 6631
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RegionID    6632 non-null   int64  
 1   SizeRank    6632 non-null   int64  
 2   zip_code    6632 non-null   int64  
 3   RegionType  6632 non-null   object 
 4   StateName   6632 non-null   object 
 5   State       6632 non-null   object 
 6   City        6572 non-null   object 
 7   Metro       6621 non-null   object 
 8   CountyName  6632 non-null   object 
 9   Jan_2023    6632 non-null   float64
 10  Feb_2023    6632 non-null   float64
 11  Mar_2023    6632 non-null   float64
 12  Apr_2023    6632 non-null   float64
 13  May_2023    6632 non-null   float64
 14  Jun_2023    6632 non-null   float64
 15  Jul_2023    6632 non-null   float64
 16  Aug_2023    6632 non-null   float64
 17  Sep_2023    6632 non-null   float64
 18  Oct_2023    6632 non-null   float64
 19  Nov_2023    6632 non-null  

In [30]:
df_zillow_data.head()

,RegionID,SizeRank,zip_code,RegionType,StateName,State,City,Metro,CountyName,Jan_2023,...,Apr_2023,May_2023,Jun_2023,Jul_2023,Aug_2023,Sep_2023,Oct_2023,Nov_2023,Dec_2023,Jan_2024
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1850.320641,...,1843.953065,1853.546220,1860.805060,1873.335787,1879.080480,1882.092604,1877.636803,1857.636589,1846.701735,1839.654960
1,61148,2,8701,zip,NJ,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County,1816.654364,...,1864.080387,1883.163864,1906.437439,1909.304801,1901.070835,1883.952296,1868.868862,1819.126778,1786.919616,1772.500000
2,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1777.611107,...,1799.232097,1803.978538,1815.603187,1824.661645,1837.338997,1837.127291,1822.343233,1809.231267,1813.118556,1830.410884
3,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,1811.758914,...,1841.080353,1859.306448,1864.888023,1860.577001,1864.587265,1861.155500,1889.310623,1879.227108,1886.003582,1881.000505
4,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,1376.028266,...,1373.506118,1378.743001,1381.945632,1405.219882,1403.055653,1409.571610,1414.197679,1418.818592,1434.125145,1430.949495


In [31]:
def setup_new_postgis_database(username, db_name):
    # Check if the database already exists
    check_db_command = rf"psql -U {username} -lqt | cut -d \| -f 1 | grep -qw {db_name}"
    db_exists = subprocess.run(check_db_command, shell=True, check=False).returncode == 0

    if not db_exists:
        # Create a new PostgreSQL database
        createdb_command = f"createdb -U {username} {db_name}"
        subprocess.run(createdb_command, shell=True, check=True)
    else:
        print(f"Database '{db_name}' already exists.")

    # Enable the PostGIS extension
    enable_postgis_command = f"psql -U {username} -d {db_name} -c 'CREATE EXTENSION IF NOT EXISTS postgis;'"
    subprocess.run(enable_postgis_command, shell=True, check=True)

    print(f"PostGIS extension enabled for database '{db_name}'.")

In [32]:
setup_new_postgis_database(DB_USER, DB_NAME)

CREATE EXTENSION
PostGIS extension enabled for database 'group8project'.


In [33]:
engine = create_engine(DB_URL)